In [ ]:
# %pip install pypdf2

In [ ]:
import tabula
import PyPDF2
import os
import json
import io
import re
import tabula
import os
import json
import fitz  # PyMuPDF
import pytesseract
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
# from openai import OpenAI
from unidecode import unidecode
from tqdm.notebook import tqdm
from PIL import Image
from unidecode import unidecode
from tqdm.notebook import tqdm

In [ ]:
client = OpenAI(
  organization='a',
  api_key = 'a'
)

In [ ]:
class leitorProcessual:
    def __init__(self):
        self.informacoes = fr"""
            processo_numero: 
            processo_unico:
            autor: (Nome de quem abriu o processo);
            estrategia: (se é defesa ou ataque)
            advogado_monitorado: 
            data_distribuicao: (data da distribuição do processo)
            data_citacao: (data do AR indicado para o réu)
            valor_acao: (valor indicado pelo autor)
            revelia:
            modalidade_pagamento: (interno)
            pagamento: (prinicipal ou complementação)
            sentenca_dados_decisao: True/False (depende se existe ou não)
            data_publicacao_sentenca: (data em que a sentença foi publicada)
            sentenca_resultado:(parcial procedência, negado, total procedência)
            sentenca_dispositivo: todas informações do dispositivo da sentença.
            recurso: True/False
            recurso_origem: autor/reu/ambos
            dispositivo_recurso: todas informações do dispositivo do recurso. 
            """
        
    def organizar_arquivos(self, pdf_file):
        page_content = {}
        pdf_reader = PyPDF2.PdfReader(pdf_file) 

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            page_content[page_num + 1] = text

        def remove_newlines(text_dict):
            return {key: value.replace("\n", "") for key, value in text_dict.items()}

        updated_dictionary = remove_newlines(page_content)

        count_per_page = {page: len(content.split()) for page, content in updated_dictionary.items()}
        # count_per_page = {page: len(content.replace(' ', '')) for page, content in updated_dictionary.items()}

        counts = list(count_per_page.values())
        percentile_50 = np.percentile(counts, 50)
        filtered_pages = {page: count for page, count in count_per_page.items() if count > percentile_50}
        lista_parse_pagina = list(filtered_pages.keys())

        return updated_dictionary, lista_parse_pagina
        
    def remover_acentos(self, texto):
        if pd.isna(texto):
            return texto  # Mantém o NaN como está
        return unidecode(texto)

    def sumarizar_pdf(self, updated_dictionary, lista_parse_pagina):
        estrutura_geral = []
        for i in tqdm(range(0,len(lista_parse_pagina)), desc='Parsing pages'):
            texto_pagina = updated_dictionary[lista_parse_pagina[i]]
            response = client.chat.completions.create(
                model='gpt-3.5-turbo-0125',
                messages=[
                    {"role":"system","content":"Você é um interpretador e organizador de dados não estruturados"},
                    {"role":"system","content": f"Dado o seguinte texto extraído de um documento na página {i}, identifique e estruture as informações"},
                    {"role":"system","content": f"Por favor, SEMPRE retorne as informações no seguinte formato e com separador '|': 'palavra-chave do campo|informação_pretendida"},
                    {"role":"system","content": f"Se não retornar encontrar a informação, colocar o texto: sem_informacao."},
                    {"role":"system","content": self.informacoes},
                    {"role":"user","content": f"Segue a página extraída: {texto_pagina}"}
                ],
                temperature=0.5,
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0
            )

            resposta = response.choices[0].message.content.strip().split('\n')
            estruturado = []
            for line in resposta:
                parts = line.split('|')
                if len(parts) == 2:
                    item = {
                        'campo': parts[0],
                        'informacao': parts[1] if parts[1] != 'sem_informacao' else np.nan,
                        'id': i
                    }
                    estruturado.append(item)
                else:
                    pass
                
            estrutura_geral.append(estruturado)

        dados_gerais = []
        for sublista in estrutura_geral:
            for dicionario in sublista:
                dados_gerais.append(dicionario)

        return dados_gerais
    
    def ajuste_dados_sumarizacao(self, pdf_file, dados_gerais):

        df_geral = pd.DataFrame(dados_gerais)
        df_geral.dropna(inplace=True)
        df_geral['informacao'] = df_geral['informacao'].apply(self.remover_acentos)
        df_geral['informacao'] = df_geral['informacao'].str.replace('R$', '', regex=False)
        contagem = df_geral.groupby(['campo', 'informacao']).size().reset_index(name='contagem')
        df_geral = df_geral.merge(contagem, on=['campo', 'informacao'], how='left')
        df_geral['informacao'] = df_geral['informacao'].str.upper().str.strip()
        df_geral.drop_duplicates(subset=['campo', 'informacao'], keep='first', inplace=True)
        df_geral.sort_values(by=['campo', 'id'], inplace=True)

        df_geral.to_excel(f'arquivo_temporario_{os.path.splitext(pdf_file)[0]}.xlsx', index=False)

        return df_geral
        

    def analise_informacoes(self, pdf_file, df_geral):
        parse_final = df_geral.to_json(orient='records', force_ascii=False)

        response = client.chat.completions.create(
            model='gpt-4-turbo',
            messages=[
                {"role":"system","content":"Você é um interpretador e organizador de dados não estruturados"},
                {"role":"system","content": f"As informações apresentada advém de um dataframe oriundo leitura de um processo judicial por meio de IA."},
                {"role":"system","content": f"A coluna campo refere-se a informação, a coluna contagem a quantas vezes aquela informação se repete, a coluna id é a primeira página de ocorrência e a coluna informacao tem o content"},
                {"role":"system","content": f"Faça uma análise criteriosa olhando a contagem e o conteúdo da coluna 'informacao' para esses dados. Conforme as várias informações, construa uma que corrobore com o conteúdo dado e descarte as que não faz ideia."},
                {"role":"system","content": f"A saída deve conter apenas este formato: campo | informacao | id"},
                {"role":"system","content": f"Preciso que evite repetições das informações, quero apenas uma linha para cada campo."},
                {"role":"user","content": f"Segue as informações em formato json: {parse_final}"}
            ],
            temperature=0.5,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )

        resposta = response.choices[0].message.content.strip().split('\n')

        estruturado = []
        for line in resposta:
            parts = line.split('|')
            if len(parts) == 3:
                item = {
                    'campo': parts[0],
                    'informacao': parts[1],
                    'id': parts[2]
                }
                estruturado.append(item)
            else:
                pass

        df = pd.DataFrame(estruturado)
        df.to_excel(f'tabela_final_{os.path.splitext(pdf_file)[0]}.xlsx', index=False)
        return df
        

In [ ]:
sumarizador = leitorProcessual()

In [ ]:
pdf_file = 'copia_integral_5010463-27.2022.8.24.0930.pdf'

In [ ]:
updated_dictionary, lista_parse_pagina = sumarizador.organizar_arquivos(pdf_file)

In [ ]:
# custo antes: 0,90
# custo depois: 

In [ ]:
dados_gerais = sumarizador.sumarizar_pdf(updated_dictionary, lista_parse_pagina)

In [ ]:
df_geral = sumarizador.ajuste_dados_sumarizacao(pdf_file,dados_gerais)

In [ ]:
df_informacoes = sumarizador.analise_informacoes(pdf_file, df_geral)

In [ ]:
df_geral

### TED e DED

In [ ]:
%pip install pytesseract PyMuPDF

In [ ]:
%pip install tabula-py[jpype]

In [ ]:
os.environ['JAVA_HOME'] = 'C:/Program Files (x86)/Java/jdk-11.0.22/bin'

In [ ]:
tabula_dfs = tabula.read_pdf('TED - ESTORNO (2).pdf', pages='all', area=[1,1,3000,3000], multiple_tables='False')

In [ ]:
dados = []

for i in range(0,len(tabula_dfs)):
    date_pattern = r'\d{2}/\d{2}/\d{4}'
    date_match = re.search(date_pattern, tabula_dfs[0]['Transferência Eletrônica Disponível - TED "IF-CLI"'].iloc[16])
    valor = tabula_dfs[i]['Unnamed: 3'].iloc[11]
    hora = tabula_dfs[i]['Unnamed: 0'].iloc[11]

    dicionario = {
        'data': date_match.group() if date_match else None,
        'hora': hora,
        'valor': valor,
        'pagina': i+1
    }

    dados.append(dicionario)

In [ ]:
pd.DataFrame(dados)

In [ ]:
# instalador do tesseract: https://github.com/UB-Mannheim/tesseract/wiki
# baixar por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata (leitura em português)

In [ ]:
tabula_dfs = tabula.read_pdf('DED.pdf', pages='all', area=[225,1,3000,3000], multiple_tables='True')

In [ ]:
tabela_concat = pd.concat(tabula_dfs, ignore_index=True)[["DT. VCTO.","PARC.","VLR. FINAL","TOT. DIVIDA","VLR.LNC./PG.","HIST. FINANC."]].dropna(subset=["DT. VCTO."])
tabela_concat = tabela_concat[tabela_concat["DT. VCTO."] != "Continua ..."]
# tabela_concat[tabela_concat['HIST. FINANC.'] == '101 - Baixa Empregador']
tabela_concat[['codigo', 'descricao']] = tabela_concat['HIST. FINANC.'].str.split(' - ', expand=True)
tabela_concat[['texto', 'valor']] = tabela_concat['VLR.LNC./PG.'].str.extract(r'(.*?)(\d+,\d+)$')
tabela_concat[tabela_concat['codigo'] == '101']

In [ ]:
tabula_dfs[1]

In [ ]:
os.environ['TESSDATA_PREFIX'] = r'C:/Program Files/Tesseract-OCR/tessdata'

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'  # Caminho para o executável do tesseract

parse_pdf = []

# Abra o arquivo PDF com o PyMuPDF
pdf_file = 'DED.pdf'
pdf = fitz.open(pdf_file)

# Itere pelas páginas e aplique OCR
for page_num in tqdm(range(len(pdf)), desc='Parsing pages'):
    # Obtenha a página
    page = pdf[page_num]

    # Renderize a página em uma imagem
    pix = page.get_pixmap()
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Extraia o texto da imagem usando OCR
    content = pytesseract.image_to_string(img, lang='por')  # Especifique o idioma com o parâmetro lang, se necessário

    dicionario = {
        'page_num': page_num + 1,
        'conteudo': content
    }

    # Faça o que você precisar com o texto
    parse_pdf.append(content)

pdf.close()

In [ ]:
pd.DataFrame(parse_pdf)[0][1]